In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Path to your dataset folder
dataset_path = 'datos/'

# Paths for the training and testing directories
train_dir = './train'
test_dir = './test'

# Loop through each folder (class) in the dataset
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    # Check if it's a directory (class folder)
    if os.path.isdir(folder_path):
        # Get all images in the folder
        images = os.listdir(folder_path)
        image_paths = [os.path.join(folder_path, img) for img in images]

        # Split the images into training and testing (80-20 split)
        train_paths, test_paths = train_test_split(image_paths, test_size=0.2)

        # Create class directories in train and test folders
        os.makedirs(os.path.join(train_dir, folder), exist_ok=True)
        os.makedirs(os.path.join(test_dir, folder), exist_ok=True)

        # Move the files into train and test directories
        for train_img in train_paths:
            shutil.move(train_img, os.path.join(train_dir, folder, os.path.basename(train_img)))

        for test_img in test_paths:
            shutil.move(test_img, os.path.join(test_dir, folder, os.path.basename(test_img)))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Paths to the train and test directories (set these to where your data is stored)
train_dir = './train'
test_dir = './test'

# Parameters
img_height = 64  # Resize the image to 64x64
img_width = 64
batch_size = 32

# Data Preprocessing with ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize images to [0, 1]
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

# Load and preprocess images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),  # Resize to 64x64
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    color_mode='grayscale'     # Grayscale images
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),  # Resize to 64x64
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

# Build the CNN Model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),  # Update input shape to match resized images
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),  # Adjusted Dense layer size after flattening
    layers.Dense(len(train_generator.class_indices), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Multi-class classification loss
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,  # You can adjust the number of epochs
    validation_data=test_generator
)

# Save the model after training
model.save('trained_model.h5')

# Save class indices from the training data generator for future use
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())  # Get class names

# Plotting the training history
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test accuracy: {test_acc}")


Found 3680 images belonging to 46 classes.
Found 920 images belonging to 46 classes.
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 15s 110ms/step - accuracy: 0.3302 - loss: 2.5229 - val_accuracy: 0.8402 - val_loss: 0.4689
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 12s 104ms/step - accuracy: 0.9277 - loss: 0.2477 - val_accuracy: 0.9924 - val_loss: 0.0357
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 12s 101ms/step - accuracy: 0.9745 - loss: 0.0888 - val_accuracy: 0.9935 - val_loss: 0.0246
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 13s 109ms/step - accuracy: 0.9845 - loss: 0.0469 - val_accuracy: 0.9957 - val_loss: 0.0136
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 17s 146ms/step - accuracy: 0.9891 - loss: 0.0360 - val_accuracy: 0.9957 - val_loss: 0.0264
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 11s 98ms/step - accuracy: 0.9910 - loss: 0.0330 - val_accuracy: 0.9946 - val_loss: 0.0267
Epoch 7/10
 56/115 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.9899 - loss: 0.0261

In [ ]:
#Guardamos el modelo 

model.save('japanese_vocab_model.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Load the trained model
model = tf.keras.models.load_model('japanese_vocab_model.h5')

# Load class indices from the training phase (you can save this during training or hardcode it)
class_names = ['aa', 'chi', 'ee']  # Replace with the actual class names from your training data

# Function to preprocess the image and make a prediction
def test_image(image_path):
    # Load the image and resize it to the model's expected input size
    img = image.load_img(image_path, target_size=(64, 64), color_mode='grayscale')
    
    # Convert the image to a numpy array
    img_array = image.img_to_array(img)
    
    # Normalize the image
    img_array = img_array / 255.0
    
    # Add an extra dimension to represent the batch size (1 image in this case)
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    predictions = model.predict(img_array)
    
    # Get the class index with the highest probability
    predicted_class_index = np.argmax(predictions)
    
    # Get the predicted class name
    predicted_class_name = class_names[predicted_class_index]
    
    # Display the image and predicted class
    plt.imshow(img, cmap='gray')
    plt.title(f'Predicted: {predicted_class_name}')
    plt.show()

    return predicted_class_name



In [26]:
image_path = './test/aa/drawing_20250805_081550.jpg'
predicted_class = test_image(image_path)
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step


AttributeError: 'Sequential' object has no attribute 'class_indices'